In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from math import sqrt
import sys, os
from contextlib import contextmanager
import matplotlib as mpl
import seaborn as sns
import sklearn
import string
import nltk
import regex
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from nltk.corpus import stopwords
from collections import Counter
import math

In [6]:
data = pd.read_csv('data_lab3/job-market.csv')
data.head(5)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
0,37404348.0,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0.0,30.0,NaN
1,37404337.0,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0.0,30.0,NaN
2,37404356.0,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0.0,30.0,NaN
3,37404330.0,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07T00:00:00.000Z,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0.0,30.0,NaN
4,37404308.0,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0.0,30.0,NaN


In [7]:
list(set(data['Classification'].values))

[nan,
 'Insurance & Superannuation',
 'Advertising, Arts & Media',
 'Manufacturing, Transport & Logistics',
 'Hospitality & Tourism',
 'Sport & Recreation',
 'Human Resources & Recruitment',
 'Science & Technology',
 'Healthcare & Medical',
 'Marketing & Communications',
 'Design & Architecture',
 'Legal',
 'Administration & Office Support',
 'Sales',
 'Retail & Consumer Products',
 'Farming, Animals & Conservation',
 'Construction',
 'Self Employment',
 'Information & Communication Technology',
 'Trades & Services',
 'Consulting & Strategy',
 'Government & Defence',
 'Education & Training',
 'Mining, Resources & Energy',
 'Accounting',
 'Call Centre & Customer Service',
 'Real Estate & Property',
 'Engineering',
 'Community Services & Development',
 'CEO & General Management',
 'Banking & Financial Services']

In [8]:
options = ['Science & Technology', 'Information & Communication Technology', 'Engineering']
filter_classification = data[data['Classification'].isin(options)]
filter_classification.head(5)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
89,37404291.0,Technical Support Engineer,FTI Consulting,2018-10-07T00:00:00.000Z,Sydney,"CBD, Inner West & Eastern Suburbs",Information & Communication Technology,Help Desk & IT Support,Act as an advisor to staff in all practice are...,NaN,0.0,30.0,NaN
96,37404325.0,Business Development Manager,TPG Telecom Limited,2018-10-07T00:00:00.000Z,Perth,"CBD, Inner & Western Suburbs",Information & Communication Technology,Sales - Pre & Post,Looking to get your foot in the door in a care...,NaN,0.0,30.0,NaN
126,37404267.0,Technician,Zoom Recruitment & Training,2018-10-07T00:00:00.000Z,Sydney,South West & M5 Corridor,Engineering,Mechanical Engineering,"This Australian Icon, connects the people of t...","<p>This Australian Icon, connects the people o...",0.0,30.0,Full Time
127,37404230.0,Systems Engineer,Humanised Group,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Information & Communication Technology,Networks & Systems Administration,Systems Engineer to work on BAU/Projects for a...,<strong>The Company</strong><br> <br> This org...,0.0,30.0,Full Time
130,37404370.0,Operations Delivery Manager,Woolworths Group,2018-10-07T00:00:00.000Z,Sydney,"CBD, Inner West & Eastern Suburbs",Information & Communication Technology,Programme & Project Management,Ready for a challenge? Opportunity to be a par...,<p> </p>\n <div></div>\n <div></...,0.0,30.0,Full Time


In [6]:
list(set(filter_classification['SubClassification'].values))

['Mechanical Engineering',
 'Networks & Systems Administration',
 'Sales - Pre & Post',
 'Architects',
 'Environmental Engineering',
 'Developers/Programmers',
 'Engineering Drafting',
 'Laboratory & Technical Services',
 'Civil/Structural Engineering',
 'Chemistry & Physics',
 'Project Engineering',
 'Project Management',
 'Testing & Quality Assurance',
 'Database Development & Administration',
 'Industrial Engineering',
 'Automotive Engineering',
 'Field Engineering',
 'Systems Engineering',
 'Quality Assurance & Control',
 'Environmental, Earth & Geosciences',
 'Programme & Project Management',
 'Electrical/Electronic Engineering',
 'Engineering - Software',
 'Telecommunications',
 'Mathematics, Statistics & Information Sciences',
 'Water & Waste Engineering',
 'Chemical Engineering',
 'Engineering - Hardware',
 'Help Desk & IT Support',
 'Management',
 'Maintenance',
 'Web Development & Production',
 'Consultants',
 'Team Leaders',
 'Technical Writing',
 'Other',
 'Biological & Bio

In [9]:
options = ['Web Development & Production', 'Developers/Programmers',
           'Mathematics, Statistics & Information Sciences', 'Testing & Quality Assurance',
           'Quality Assurance & Control', 'Help Desk & IT Support', 'Project Engineering',
           'Engineering - Software', 'Engineering - Hardware', 'Business/Systems Analysts',
           'Networks & Systems Administration', 'Process Engineering', 'Database Development & Administration',
           'Telecommunications', 'Security', 'Systems Engineering']
filter_subclassification = filter_classification[filter_classification['SubClassification'].isin(options)]
filter_subclassification = filter_subclassification.dropna(subset=['FullDescription'])
filter_subclassification.head(5)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
127,37404230.0,Systems Engineer,Humanised Group,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Information & Communication Technology,Networks & Systems Administration,Systems Engineer to work on BAU/Projects for a...,<strong>The Company</strong><br> <br> This org...,0.0,30.0,Full Time
133,37404174.0,Technical Support Executive - $70K + Super - C...,Command Group,2018-10-07T00:00:00.000Z,Sydney,"CBD, Inner West & Eastern Suburbs",Information & Communication Technology,Help Desk & IT Support,Do you have strong technical support skills? L...,<div>\n <ul>\n <li>Great s...,0.0,30.0,Full Time
143,37404368.0,Senior Manager - Security Incident Response Ma...,Woolworths Group,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Information & Communication Technology,Security,Senior Manager - Security Incident Management ...,<p> </p>\n <div></div>\n <div></...,0.0,30.0,Full Time
146,37404367.0,Security Incident Response Specialist,Woolworths Group,2018-10-07T00:00:00.000Z,Sydney,Parramatta & Western Suburbs,Information & Communication Technology,Security,Security Incident Response Specialist Location...,<p> </p>\n <div></div>\n <div></...,0.0,30.0,Full Time
148,37404369.0,Cloud Security Architect,Woolworths Group,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Information & Communication Technology,Security,About Woolworths IT team As Australia's bigg...,<p> </p>\n <div></div>\n <div></...,0.0,30.0,Full Time


In [10]:
raw_description = list(filter_subclassification['FullDescription'].values)
raw_description[0]

'<strong>The Company</strong><br> <br> This organisation is well-established, with a vast market presence. They have a proud history of having delivered cutting edge solutions and are strongly committed to their customer needs, the business is\n          genuinely at the forefront of its field.<br> <br> <strong>The role</strong><br> Coming in as a Senior within the team you will be responsible for their main accounts, this role will see a main focus on BAU where you will also be required to\n          touch on projects (delivering project work based off a project plan). The technologies you will have experience across include;<br>\n          <ul>\n            <li>Hyper-V (ideal).</li>\n            <li>VMware.</li>\n            <li>Server 2012/2016.</li>\n            <li>Exchange, Office 365 and PowerShell.</li>\n            <li>HP, Dell, IBM, Ubiquiti &amp; Cyberoam/Sophos.</li>\n            <li>IP Networking LAN &amp; WAN (SSL/DNS/VPN).</li>\n            <li>Microsoft Server, AD, Grou

In [11]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def clean_html(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    #cleantext = re.sub('\s+',' ', cleantext)
    return cleantext

description = [clean_html(text) for text in raw_description]
description[0]

'The Company  This organisation is well-established, with a vast market presence. They have a proud history of having delivered cutting edge solutions and are strongly committed to their customer needs, the business is\n          genuinely at the forefront of its field.  The role Coming in as a Senior within the team you will be responsible for their main accounts, this role will see a main focus on BAU where you will also be required to\n          touch on projects (delivering project work based off a project plan). The technologies you will have experience across include;\n          \n            Hyper-V (ideal).\n            VMware.\n            Server 2012/2016.\n            Exchange, Office 365 and PowerShell.\n            HP, Dell, IBM, Ubiquiti  Cyberoam/Sophos.\n            IP Networking LAN  WAN (SSL/DNS/VPN).\n            Microsoft Server, AD, Group Policy.\n            Most of this work will be server related however you will have the opportunity to be upskilled to run UTMs 

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 1), min_df = 1, stop_words = 'english', max_features = 500)

features = tfidf.fit(description)
corpus_tf_idf = tfidf.transform(description)

sum_words = corpus_tf_idf.sum(axis = 0)
words_freq = [(word, sum_words[0, idx]) for word, idx in tfidf.vocabulary_.items()]
top20 = sorted(words_freq, key=lambda x: x[1], reverse=True)[:20]
for keyword in top20:
    print(keyword[0], keyword[1])

experience 28.48592927646742
support 18.777025978019402
team 18.72130763696386
work 18.527651418960918
business 18.316274355476608
skills 18.06646026541815
role 16.73847454334689
development 15.421064973762588
service 15.182483609751388
working 13.51932688855057
customer 13.485757650656574
technical 13.027839180238235
ability 12.562519609860782
security 12.36468405595014
systems 12.223606808272264
strong 11.95007490961257
services 11.431915967164624
knowledge 11.367494427708966
management 11.208679284575359
software 11.1645060056297


In [13]:
query = "experience"
new_features = tfidf.transform([query])
cosine_similarities = linear_kernel(new_features, corpus_tf_idf).flatten()
related_docs_indices = cosine_similarities.argsort()[::-1]

topk = 20
print('Top-{0} documents'.format(topk))
for i in range(topk):
    print(i, description[related_docs_indices[i]])

Top-20 documents
0 About the position
        This is your opportunity to influence the future of the financial world and work on some new and exciting technologies within Blockchain in an expanding and fun Start-Up environment.
        Your chance to problem solve with some of the smartest Engineers in Sydney on issues that will shape the future of the FinTech Space.
        We are looking for Strong Thinkers, not just coders. People who can think outside the box and have a true passion for Technology and treat it as more than just a job.
        Experience level isn't an issue here, more willingness to learn, ability to communicate strongly and to problem solve effectively.
        
        Skills Required
        
          Sound knowledge of Development in Java8
          Experience with the Spring eco-system or an alternative
          Development experience of Rest APIs
          Microservices Architecture experience
          Agile, CI/CD environment experience
          Relevan